In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
X_train_tensor = torch.from_numpy(X_train)
Y_train_tensor = torch.from_numpy(Y_train)
X_test_tensor  = torch.from_numpy(X_test)
Y_test_tensor  = torch.from_numpy(Y_test)

train_tensor_dataset = torch.utils.data.TensorDataset(X_train_tensor, Y_train_tensor)
test_tensor_dataset  = torch.utils.data.TensorDataset(X_test_tensor, Y_test_tensor)

data_loader_train = torch.utils.data.DataLoader(dataset=train_tensor_dataset, 
                                          batch_size=10, 
                                          shuffle=True,
                                          num_workers=4)

data_loader_test = torch.utils.data.DataLoader(dataset=test_tensor_dataset, 
                                          batch_size=100, 
                                          shuffle=False,
                                          num_workers=4)

In [ ]:
# Neural Network Model
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = F.dropout(x, p=0.1, training=True)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = F.dropout(x, p=0.1, training=True)
        x = self.fc2(x)
        return x

In [ ]:
model = Model(input_size=1, hidden_size=10, output_size=1)

optimizer = torch.optim.SGD(model.parameters(),
                            lr=0.001,
                            weight_decay=1.0778889384649698e-06,
                            momentum=0.9)

loss_criterion = torch.nn.MSELoss()

In [ ]:
train_loss = []
test_loss  = []

# Train the Model
for epoch in range(40):
    #print(f'Epoch {epoch}/40')
    
    #train
    batchloss_train = []
    for X_train_batch, Y_train_batch in data_loader_train:  
        # Convert torch tensor to Variable
        X_train_tensor = Variable(X_train_batch).float()
        Y_train_tensor = Variable(Y_train_batch).float()
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        predicted_Y = model(X_train_tensor)
        trainloss = loss_criterion(predicted_Y, Y_train_tensor)
        trainloss.backward()
        optimizer.step()
        batchloss_train.append(trainloss.data[0])
        
    #scheduler.step()
    train_loss.append(np.mean(batchloss_train))
    
print('Train RMSE: {}'.format(np.sqrt(train_loss[-1])))

In [ ]:
#Test 
N_forward = 100
Y_hat_matrix = np.zeros([N_forward, 100])
for i in range(N_forward):
    for X_test_batch, Y_test_batch in data_loader_test:
        Y_hat = model(Variable(X_test_batch).float())
    Y_hat_matrix[i,:] = Y_hat.data.numpy().T